# Capstone project 1: Predicting the number of visits in US neighborhoods (continued)
# Machine Learning

**Table of Content**
- Introduction
- Machine Learning
- Conclusions

## Introduction
The goal of the machine learning part of the project is to ... 

## Machine Learning

### Step 1: import modules and files

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import operator
import geopandas as gpd
import descartes
import ast
from shapely.geometry import Point , Polygon
from collections import Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing
import time
from math import radians, cos, sin, atan2, sqrt
import random
import hdbscan
%matplotlib inline

/Users/colinechabloz/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/colinechabloz/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
file_path = '/Users/colinechabloz/Desktop/Datasets/Cluster/'
x_cols1_df_n = 'x_cols1_df.csv'

In [3]:
#Load several files from csv
x_cols1_df = pd.read_csv((file_path+x_cols1_df_n), dtype={'census_block_group':str})

In [4]:
x_cols1_df.head()

,Unnamed: 0,census_block_group,raw_visit_count,raw_visitor_count,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,latitude,longitude,...,vis_home_cbgs_count,vis_home_most_visits_from,vis_home_least_visits_from,avg_vis_home_visitors,visitor_work_cbgs,vis_work_cbgs_count,vis_work_most_visits_from,vis_work_least_visits_from,avg_vis_work_visitors,Cluster
0,0,010059508001,75122.0,18314.0,194724.0,"['chick-fil-a', 'mcdonalds', 'marathonpetroleu...","['walmart', 'mcdonalds', 'dollargeneral', 'chi...","['crossfit', 'healthmart', 'coldwellbanker']",31.920410,-85.175741,...,14,1.005950e+10,1.067030e+10,76.285714,"{'010059501003': 109, '010810407002': 62, '010...",4,1.005950e+10,1.069040e+10,70.000000,2
1,1,010090506011,95649.0,38942.0,120587.0,"['shelloil', 'mcdonalds', 'chick-fil-a', 'chev...","['walmart', 'mcdonalds', 'shelloil', 'chick-fi...",[],33.863884,-86.903514,...,69,1.073011e+10,1.009051e+10,80.623188,"{'010890111001': 271, '010730045001': 269, '01...",41,1.089011e+10,1.077011e+10,93.756098,2
2,2,010479572002,14009.0,3039.0,67774.0,['dollargeneral'],"['walmart', 'dollargeneral', 'mcdonalds', 'che...",['dollargeneral'],32.357337,-87.014004,...,2,1.047957e+10,1.047957e+10,63.500000,{'010479567021': 52},1,1.047957e+10,1.047957e+10,52.000000,0
3,3,010690404001,128169.0,25418.0,42684.0,"['chick-fil-a', 'samsclub', 'dollargeneral', '...","['walmart', 'dollargeneral', 'mcdonalds', 'mar...","['chick-fil-a', 'samsclub', 'olivegarden', 'mc...",31.237327,-85.426331,...,92,1.069040e+10,1.069040e+10,99.000000,"{'010690402024': 313, '010690415004': 203, '01...",23,1.069040e+10,1.031011e+10,108.173913,1
4,4,010730112101,51453.0,9499.0,18878.0,"['chevron', 'daylightdonuts', 'walmart']","['walmart', 'chevron', 'dollargeneral', 'shell...","['chevron', 'crossfit']",33.671800,-86.690610,...,37,1.009051e+10,1.073012e+10,81.513514,"{'010730045001': 140, '010730027001': 123, '01...",10,1.073005e+10,1.073002e+10,86.500000,2


In [5]:
x_cols1_df.shape

(220240, 83)

- filter cluster = 0 
- split train (70%) and test (20%) and validation (10%) to be used at the end to perform new predictions and create plots other than when training/testing the model
- with train set: regression y .. X ... parameters (decide metrics: eg; minimum root mean squared error)
- with test set : predict (same regression)
- measure performance of regression with train and the performance of regression with test 
- compare performances and evaluate the quality of model

In [6]:
#cluster 0
x_cols1_df_0 = x_cols1_df[x_cols1_df['Cluster'] == 0]

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [9]:
y = x_cols1_df_0["raw_visit_count"]

In [29]:
y.describe()

count    150075.000000
mean      23108.538444
std       12194.861217
min          60.000000
25%       13431.000000
50%       21445.000000
75%       31528.000000
max       71085.000000
Name: raw_visit_count, dtype: float64

In [10]:
x_cols = np.array(x_cols1_df_0[['12am','1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', '12pm', '1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']])

In [11]:
X = np.asmatrix(x_cols)

In [12]:
X

matrix([[ 807.,  790.,  796., ..., 2076., 1707., 1536.],
        [2459., 2397., 2363., ..., 3712., 3138., 2893.],
        [ 673.,  554.,  484., ..., 2942., 2497., 2257.],
        ...,
        [ 758.,  735.,  735., ...,  952.,  792.,  650.],
        [ 364.,  350.,  358., ...,  375.,  336.,  285.],
        [ 599.,  587.,  563., ...,  943.,  938.,  846.]])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
reg_all.coef_

array([ 1.36686868e-14, -1.95399252e-14, -2.04558592e-14,  1.05471187e-15,
        8.65973959e-15,  5.10702591e-15, -3.55271368e-15,  5.10702591e-15,
       -4.32986980e-15,  1.99840144e-15,  5.57887070e-15, -8.88178420e-15,
        3.88578059e-15,  4.85722573e-15,  1.77635684e-15,  3.30291350e-15,
        2.13717932e-15,  3.49720253e-15, -8.32667268e-16,  2.41473508e-15,
        2.02615702e-15,  3.10862447e-15,  5.85642645e-15,  2.02615702e-15,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00])

Variables "Monday" to "Sunday" (last 7) seem to be perfectly correlated with independent variable y. Why?

In [32]:
x_cols1_df_0.corr()

,Unnamed: 0,raw_visit_count,raw_visitor_count,distance_from_home,latitude,longitude,12am,1am,2am,3am,...,Sunday(ratio),vis_home_cbgs_count,vis_home_most_visits_from,vis_home_least_visits_from,avg_vis_home_visitors,vis_work_cbgs_count,vis_work_most_visits_from,vis_work_least_visits_from,avg_vis_work_visitors,Cluster
Unnamed: 0,1.000000,-0.007634,-0.006120,-0.002652,0.003386,-0.001816,-0.005490,-0.005306,-0.005005,-0.004648,...,0.002052,-0.009628,-0.002428,-0.001899,-0.006534,-0.008793,-0.002987,-0.002728,-0.006244,NaN
raw_visit_count,-0.007634,1.000000,0.829881,-0.014428,-0.062490,-0.025764,0.509910,0.468764,0.449369,0.444709,...,-0.187185,0.685114,-0.045954,-0.045185,0.525128,0.662640,-0.035988,-0.036478,0.534330,NaN
raw_visitor_count,-0.006120,0.829881,1.000000,0.074486,-0.055992,-0.032122,0.319085,0.276202,0.255060,0.248005,...,-0.005523,0.493739,-0.064262,-0.062324,0.314565,0.619569,-0.060177,-0.059680,0.395088,NaN
distance_from_home,-0.002652,-0.014428,0.074486,1.000000,-0.014804,-0.110682,-0.057406,-0.055922,-0.054630,-0.052013,...,0.197401,-0.068800,0.007815,0.010160,-0.079003,-0.080263,0.015863,0.016832,-0.103626,NaN
latitude,0.003386,-0.062490,-0.055992,-0.014804,1.000000,-0.000345,-0.099975,-0.098036,-0.098428,-0.097045,...,-0.001195,-0.170210,0.195744,0.194657,-0.097734,-0.098543,0.214596,0.214950,-0.060373,NaN
longitude,-0.001816,-0.025764,-0.032122,-0.110682,-0.000345,1.000000,-0.050230,-0.046404,-0.038503,-0.026310,...,-0.043861,-0.080908,0.318204,0.317671,-0.091204,-0.113477,0.308068,0.307741,-0.088545,NaN
12am,-0.005490,0.509910,0.319085,-0.057406,-0.099975,-0.050230,1.000000,0.996681,0.992769,0.988703,...,0.041953,0.327984,-0.022600,-0.022786,0.447638,0.189930,-0.017072,-0.017409,0.297969,NaN
1am,-0.005306,0.468764,0.276202,-0.055922,-0.098036,-0.046404,0.996681,1.000000,0.997928,0.994779,...,0.047562,0.304182,-0.018490,-0.018722,0.434020,0.158282,-0.013426,-0.013851,0.276437,NaN
2am,-0.005005,0.449369,0.255060,-0.054630,-0.098428,-0.038503,0.992769,0.997928,1.000000,0.997594,...,0.048780,0.294498,-0.011007,-0.011305,0.427239,0.144685,-0.005925,-0.006378,0.265652,NaN
3am,-0.004648,0.444709,0.248005,-0.052013,-0.097045,-0.026310,0.988703,0.994779,0.997594,1.000000,...,0.051075,0.291279,-0.006856,-0.007087,0.424654,0.136141,-0.002106,-0.002588,0.257989,NaN


In [20]:
y_pred = reg_all.predict(X_test)

In [21]:
reg_all.score(X_test, y_test)

1.0

In [23]:
print("R^2: {}".format(reg_all.score(X_test, y_test)))

R^2: 1.0


In [27]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))

In [28]:
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 5.968098552999438e-11


<b> Comparing orginal dataset vs dataset currently used for prediction</b>

In [33]:
file_path_or = '/Users/colinechabloz/Desktop/Datasets/'
x_cols1_df_n_or = 'cbg_patterns.csv'

In [35]:
original_dataset = pd.read_csv((file_path_or+x_cols1_df_n_or), nrows=10)

In [36]:
original_dataset.head()

,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day
0,10059508001,1538352000,1541030400,75122,18314,"{""010059501003"":127,""010059509001"":111,""010059...","{""010059501003"":109,""010810407002"":62,""0108104...",194724,"[""Chick-fil-A"",""mcdonalds"",""Marathon Petroleum...","[""walmart"",""mcdonalds"",""Dollar General"",""Chick...","[""CrossFit"",""Health Mart"",""Coldwell Banker""]","[2617,2457,2403,2519,2646,3007,3886,7566,5508,...","{""Monday"":12000,""Tuesday"":12224,""Wednesday"":10..."
1,10090506011,1538352000,1541030400,95649,38942,"{""010730113021"":210,""010090506022"":205,""010090...","{""010890111001"":271,""010730045001"":269,""010439...",120587,"[""Shell Oil"",""mcdonalds"",""Chick-fil-A"",""Chevron""]","[""walmart"",""mcdonalds"",""Shell Oil"",""Chick-fil-...",[],"[6556,6325,6222,6355,6586,7350,8568,8099,7378,...","{""Monday"":12125,""Tuesday"":12984,""Wednesday"":12..."
2,10479572002,1538352000,1541030400,14009,3039,"{""010479567011"":67,""010479567021"":60}","{""010479567021"":52}",67774,"[""Dollar General""]","[""walmart"",""Dollar General"",""mcdonalds"",""Chevr...","[""Dollar General""]","[807,790,796,786,851,951,1134,1797,1355,1241,1...","{""Monday"":2314,""Tuesday"":2340,""Wednesday"":2195..."
3,10690404001,1538352000,1541030400,128169,25418,"{""010690402013"":370,""010690402011"":322,""010690...","{""010690402024"":313,""010690415004"":203,""010450...",42684,"[""Chick-fil-A"",""Sam's Club"",""Dollar General"",""...","[""walmart"",""Dollar General"",""mcdonalds"",""Marat...","[""Chick-fil-A"",""Sam's Club"",""Olive Garden"",""mc...","[2121,1828,1784,1704,1861,2373,3730,7497,7093,...","{""Monday"":21141,""Tuesday"":21143,""Wednesday"":17..."
4,10730112101,1538352000,1541030400,51453,9499,"{""010090507001"":183,""010730113021"":167,""010730...","{""010730045001"":140,""010730027001"":123,""010730...",18878,"[""Chevron"",""Daylight Donuts"",""walmart""]","[""walmart"",""Chevron"",""Dollar General"",""Shell O...","[""Chevron"",""CrossFit""]","[3804,3716,3686,3672,3735,4115,4855,5946,4526,...","{""Monday"":8402,""Tuesday"":8414,""Wednesday"":8550..."


In [39]:
original_dataset.loc[original_dataset['census_block_group']==10059508001, ['popularity_by_hour','popularity_by_day']]

,popularity_by_hour,popularity_by_day
0,"[2617,2457,2403,2519,2646,3007,3886,7566,5508,...","{""Monday"":12000,""Tuesday"":12224,""Wednesday"":10..."


In [45]:
x_cols1_df.loc[x_cols1_df['census_block_group']=='010059508001', ['12am']]

,12am
0,2617.0
